In [28]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# 2. Analyzing the Data

It has monthly data on financial indexes and ETFs from Aug 2011 through May 2025.

## 1.

For the series in the “hedge fund series” tab, report the following summary statistics:

- mean

- volatility

- Sharpe ratio

Annualize these statistics.

In [26]:
hedgeFund = pd.read_excel("../../data/proshares_analysis_data.xlsx", sheet_name = "hedge_fund_series")
hedgeFund = hedgeFund.rename(columns = {"Unnamed: 0": "Date"}).set_index("Date")
merrillFactors = pd.read_excel("../../data/proshares_analysis_data.xlsx", sheet_name = "merrill_factors")
merrillFactors = merrillFactors.rename(columns = {"Unnamed: 0": "Date"}).set_index("Date")
hedgeFund.head()

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
Date,,,,,
2011-08-31,-0.032149,-0.025588,-0.025689,-0.027033,-0.006492
2011-09-30,-0.038903,-0.032414,-0.032593,-0.032466,-0.022142
2011-10-31,0.026858,0.043593,0.043320,0.050531,0.025244
2011-11-30,-0.013453,-0.012142,-0.012431,-0.028608,-0.007965
2011-12-31,-0.004479,0.001938,0.001796,0.012874,0.001818


In [46]:
mean, std = hedgeFund.mean() * 12, hedgeFund.std() * np.sqrt(12)
sharpe = mean / std

result = pd.DataFrame({
    "Mean": mean, 
    "Std": std, 
    "Sharpe Ratio": sharpe
})
result.T

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
Mean,0.051279,0.038535,0.036526,0.026880,0.028811
Std,0.058796,0.055240,0.055102,0.057388,0.049823
Sharpe Ratio,0.872151,0.697589,0.662880,0.468396,0.578275


### 2.

For the series in the “hedge fund series” tab, calculate the following statistics related to tail-risk.

- Skewness

- Excess Kurtosis (in excess of 3)

- VaR (.05) - the fifth quantile of historic returns

- CVaR (.05) - the mean of the returns at or below the fifth quantile

- Maximum drawdown - include the dates of the max/min/recovery within the max drawdown period.

There is no need to annualize any of these statistics.

In [ ]:
def CalcMaxDrawdown(returns):
    cumulative = (1 + returns).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    max_drawdown = drawdown.min()
    return max_drawdown

In [45]:
skew, kurt = hedgeFund.skew(), hedgeFund.kurtosis()
var = hedgeFund.quantile(0.05)
cvar = hedgeFund[hedgeFund <= var].mean()
maxDrawdown = CalcMaxDrawdown(hedgeFund)

result2 = pd.DataFrame({
    "Skew": skew,
    "Kurtosis": kurt,
    "VaR": var,
    "CVaR": cvar,
    "Max Drawdown": maxDrawdown
})
result2.T

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
Skew,-0.948272,-0.289982,-0.273477,-0.274904,-0.433508
Kurtosis,5.657351,1.630917,1.589790,1.776481,1.449184
VaR,-0.024008,-0.027002,-0.026970,-0.029919,-0.017171
CVaR,-0.035992,-0.034993,-0.034937,-0.036831,-0.030995
Max Drawdown,-0.115473,-0.124302,-0.124388,-0.140720,-0.137716


### 3.

For the series in the “hedge fund series” tab, run a regression of each against SPY (found in the “merrill factors” tab.) Include an intercept. Report the following regression-based statistics:

- Market Beta

$$
\~r_{i, t} = \alpha + \beta * \~r_{SPY, t} + \epsilon_{t}
$$

- Treynor Ratio

$$
Treynor\ Ratio = \frac{\~\mu}{\beta}
$$

- Information Ratio

$$
Information\ Ratio = \frac{\alpha}{\sigma_{\epsilon}}
$$

Annualize these three statistics as appropriate.

In [ ]:
sp500 = merrillFactors["SPY US Equity"]
riskFreeRate = merrillFactors["USGG3M Index"] # US Generic Govt 3 Mth
sp500ExcessReturn = sp500 - riskFreeRate
marketBeta, treynorRatio, infoRatio = list(), list(), list()

# Regression
x = sm.add_constant(sp500ExcessReturn)
for col in hedgeFund.columns:
    y = hedgeFund[col].ffill() - riskFreeRate
    model = sm.OLS(y, x).fit()
    
    alpha, beta = model.params
    epsilon = model.resid.std()
    
    treynor = y.mean() / beta
    info = alpha / epsilon

    marketBeta.append(beta)
    treynorRatio.append(treynor)
    infoRatio.append(info)

result3 = pd.DataFrame({
    "Beta":  marketBeta,
    "Treynor Ratio": treynorRatio,
    "Information Ratio": infoRatio
}, index = hedgeFund.columns)
result3.T

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
Beta,0.346671,0.341857,0.340939,0.350024,0.300085
Treynor Ratio,0.009002,0.005716,0.005240,0.002808,0.003811
Information Ratio,-0.069052,-0.239243,-0.260637,-0.351231,-0.299394


### 4.
Discuss the previous statistics, and what they tell us about…

- the differences between SPY and the hedge-fund series?

- which performs better between HDG and QAI.
  - QAI have higher return mean, lower return std

- whether HDG and the ML series capture the most notable properties of HFRI.